In [39]:
imports_path = ".\\imports.ipynb"
tableGAN_path = ".\\tableGAN.ipynb"
utils_path = ".\\utils.ipynb"

%run "$imports_path"

In [40]:
%run "$utils_path"
%run "$tableGAN_path"

image_dir = ".\\Images"
model_dir = ".\\Model\\tableGAN_ckpt\\compare_pred_on_synthetic_adult\\"
dataset_dir = ".\\Datasets\\"
dataset_train_path = os.path.join(dataset_dir, "df_adult_edited_train.csv")
dataset_test_path = os.path.join(dataset_dir, "df_adult_edited_test.csv")
dataset_gen_dir = os.path.join(dataset_dir, "Generated_for_pred_eval_adult//")

data_train = pd.read_csv(dataset_train_path)
data_test = pd.read_csv(dataset_test_path)
discrete_columns = data_train.columns[data_train.dtypes == "object"]
response_col="income"
categories="auto"

In [64]:
X_train = data_train.iloc[:,data_train.columns != response_col]
Y_train = data_train[response_col]
X_test = data_test.iloc[:,data_train.columns != response_col]
Y_test = data_test[response_col]


columns_discrete_bool = [b in ["object", "category"] for b in X_train.dtypes]
discrete_columns_of_X = X_train.columns[columns_discrete_bool]
numeric_columns_of_X = X_train.columns[np.logical_not(columns_discrete_bool)]

X_train[discrete_columns_of_X].astype("category")
X_test[discrete_columns_of_X].astype("category")

label_encoder = LabelEncoder()
Y_train = label_encoder.fit_transform(Y_train)
Y_test = label_encoder.transform(Y_test)

#categories = OneHotEncoder().fit(X_train[discrete_columns_of_X]).categories_
oh_encoder = OneHotEncoder(categories = categories, sparse = False)
X_train_discr = oh_encoder.fit_transform(X_train[discrete_columns_of_X])
X_train = np.concatenate((X_train[numeric_columns_of_X].to_numpy(),
                          oh_encoder.fit_transform(X_train[discrete_columns_of_X])),
                         axis = 1)
X_test = np.concatenate((X_test[numeric_columns_of_X].to_numpy(),
                          oh_encoder.transform(X_test[discrete_columns_of_X])),
                         axis = 1)
X_train

array([[3.90000e+01, 7.75160e+04, 1.30000e+01, ..., 1.00000e+00,
        0.00000e+00, 1.00000e+00],
       [5.00000e+01, 8.33110e+04, 1.30000e+01, ..., 1.00000e+00,
        0.00000e+00, 1.00000e+00],
       [3.80000e+01, 2.15646e+05, 9.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 1.00000e+00],
       ...,
       [5.80000e+01, 1.51910e+05, 9.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [2.20000e+01, 2.01490e+05, 9.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 1.00000e+00],
       [5.20000e+01, 2.87927e+05, 9.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00]])

In [80]:
X_train_small, X_validation, Y_train_small,  Y_validation = train_test_split(X_train, Y_train)

In [83]:
np.log(0.5)

-0.6931471805599453

In [166]:
# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe


space={"eta": hp.uniform("eta", 0, 0.5),
       'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 0,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
       "max_delta_step" : hp.uniform("max_delta_step", 0, 10),
       "subsample" : hp.uniform("subsample", 0.5, 1),
        'n_estimators': hp.quniform("n_estimators", 100, 500, 1),
        'seed': 0
    }
space2 = {
    "eta": hp.uniform("eta", 0, 0.5),
}

In [167]:
def objective(space):
    clf= XGBClassifier(
        eta = space["eta"], 
        n_estimators = int(space['n_estimators']), max_depth = int(space['max_depth']), gamma = space['gamma'],
        reg_alpha = int(space['reg_alpha']), reg_lambda =int(space['reg_lambda']), 
        min_child_weight=int(space['min_child_weight']), colsample_bytree=int(space['colsample_bytree']),
        subsample=space['subsample'])
    
    evaluation = [( X_train_small, Y_train_small), ( X_validation, Y_validation)]
    
    clf.fit(X_train_small, Y_train_small,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_validation)
    accuracy = accuracy_score(Y_validation, pred>0.5)
    return {'loss': -accuracy, 'status': STATUS_OK }

def objective2(space):
    clf= XGBClassifier(eta = int(space["eta"]))
    
    evaluation = [( X_train_small, Y_train_small), ( X_validation, Y_validation)]
    
    clf.fit(X_train_small, Y_train_small,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_validation)
    accuracy = accuracy_score(Y_validation, pred>0.5)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [168]:
trials = Trials()

best_hyperparams = fmin(fn = objective2,
                        space = space2,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

100%|█████████████████████████████████████████████| 100/100 [00:23<00:00,  4.28trial/s, best loss: -0.7493701100649781]


In [169]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

The best hyperparameters are :  

{'eta': 0.43784318467350997}


In [170]:
best_hyperparams

{'eta': 0.43784318467350997}

In [171]:
clf_best= XGBClassifier(
                    n_estimators = int(best_hyperparams['n_estimators']), max_depth = int(best_hyperparams['max_depth']), gamma = best_hyperparams['gamma'],
                    reg_alpha = int(best_hyperparams['reg_alpha']),min_child_weight=int(best_hyperparams['min_child_weight']),
                    colsample_bytree=int(best_hyperparams['colsample_bytree']))

KeyError: 'n_estimators'

In [174]:
clf_best= XGBClassifier(
                    eta=best_hyperparams['eta'])

In [175]:
clf_best.fit(X_train, Y_train)

[20:50:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eta=0.43784318467350997, gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.437843174,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [176]:
pred = clf_best.predict(X_test)
accuracy = accuracy_score(Y_test, pred>0.5)
auc = roc_auc_score(Y_test, clf_best.predict_proba(X_test)[:,1])
print("Accurcay:", accuracy)
print("AUC:", auc)

Accurcay: 0.8642762284196547
AUC: 0.9210870289303388


In [143]:
clf = XGBClassifier(
        tree_method="gpu_hist", enable_categorical=False, use_label_encoder=False, eval_metric = "logloss"
    )
# X is the dataframe we created in previous snippet
clf.fit(X_train, Y_train)
# Must use JSON for serialization, otherwise the information is lost
clf.save_model("categorical-model.json")

accuracy = accuracy_score(Y_test, clf.predict(X_test))
auc = roc_auc_score(Y_test, clf.predict_proba(X_test)[:,1])
print("Accurcay:", accuracy)
print("AUC:", auc)

Accurcay: 0.8658034528552457
AUC: 0.9221433431671108
